# 2019 Novel Coronavirus (SARS-CoV-2) and COVID-19 Unpivoted Data

The following script takes data from the repository of the 2019 Novel Coronavirus Visual Dashboard operated by Johns Hopkins University's Center for Systems Science and Engineering (JHU CSSE). It will apply necessary cleansing/reformatting to make it use in traditional relational databases and data visualization tools.

In [ ]:
import pandas as pd
import os
from datetime import datetime
import pycountry
from copy import deepcopy

In [ ]:
# papermill parameters
output_folder = "../output/"

Data downloaded directly from Johns Hopkins git repository, located at: https://github.com/CSSEGISandData/COVID-19. Their repository has three different CSV files – one each for `confirmed`, `deaths` and `recovered` data. The data is keyed into an array of `pandas` `DataFrame`s.

In [ ]:
confirmed = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv",keep_default_na=False)
deaths = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv",keep_default_na=False)
recovered = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv",keep_default_na=False)

confirmed["Case_Type"] = "Confirmed"
deaths["Case_Type"] = "Deaths"
recovered["Case_Type"] = "Recovered"

key_columns = ["Country/Region",
               "Province/State",
               "Lat",
               "Long",
               "Case_Type"]

data = [confirmed, deaths, recovered]

The original dataset stores the number of `Cases` for a given day in columns. 
This is not useful for reporting, thus we move these date columns to rows:

In [ ]:
def unpivot(df):
    # unpivot all non-key columns
    melted = df.melt(id_vars=key_columns, var_name="Date", value_name="Cases")
    # change our new Date field to Date type
    melted["Date"]= pd.to_datetime(melted["Date"]) 
    
    return melted

unpivoted_data = list(map(unpivot, data))

## Data Quality

We are recombining the data set first to use county-level data aggregates before 09 March and state-level data thereafter. Because the data for `US-VI` (U.S. Virgin Islands) still contains a comma in data after March 10, we are executing a substitution before filtering.

In [ ]:
def drop_incorrect_county_state_data(df):
    stateBeforeMarch9th = df[ (df["Date"] <= "2020-03-09") & (df["Country/Region"] == "US") & (df["Province/State"].str.contains(",") == False) ].index
    countyAfterMarch10th = df[ (df["Date"] > "2020-03-09") & (df["Country/Region"] == "US") & df["Province/State"].str.contains(",") ].index

    return df.drop(stateBeforeMarch9th).drop(countyAfterMarch10th)

unpivoted_data = [df.replace({"Virgin Islands, U.S.": "Virgin Islands"}) for df in unpivoted_data]
unpivoted_data = [drop_incorrect_county_state_data(df) for df in unpivoted_data]

We normalize data on the Virgin Islands and Washington D.C. in the following step.

In [ ]:
locality_replacements = {"Washington, D.C.": "District of Columbia",
                         "Virgin Islands, U.S.": "Virgin Islands, VI"}

def replace_localities(df):
    return df.replace(locality_replacements)

unpivoted_data = [replace_localities(df) for df in unpivoted_data]

Next, we resolve the geographies of U.S. counties to their respective states (`Province/State`): 

In [ ]:
subdivisions = {i.name: i.code for i in pycountry.subdivisions.get(country_code="US")}
abbreviations = {subdivisions[k]: k for k in subdivisions}

def resolve_US_geography(row):
    county, state = row["Province/State"].split(", ")
    state.replace("D.C.", "DC")
    row["Province/State"] = abbreviations["US-" + state.strip()]
    return row
        
def resolve_geography_df(df):
    return df.apply(lambda row: resolve_US_geography(row) if row["Country/Region"] == "US" and row["Province/State"] not in list(subdivisions.keys()) and ", " in row["Province/State"] else row, axis="columns")

unpivoted_data = [resolve_geography_df(df) for df in unpivoted_data]

A number of states have inconsistent naming or special characters, such as `Taiwan*`. These are normalised through a replacement `dict`. Data is then aggregated for each division by date and case type.

In [ ]:
changed_names = {
    "Holy See": "Vatican City",
    "Hong Kong SAR": "Hong Kong",
    "Iran (Islamic Republic of)": "Iran",
    "Macao SAR": "Macau",
    "Republic of Korea": "South Korea",
    "Republic of Moldova": "Moldova",
    "Russian Federation": "Russia",
    "Saint Martin": "St. Martin",
    "Taipei and environs": "Taiwan",
    "Viet Nam": "Vietnam",
    "occupied Palestinian territory": "Palestine",
    "Taiwan*": "Taiwan",
    "Congo (Brazzaville)": "Republic of the Congo"
}


for idx,df in enumerate(unpivoted_data):
    df["Country/Region"] = df["Country/Region"].replace(changed_names)
    df["Cases"] = df["Cases"].replace('',0).astype(int)
        
    unpivoted_data[idx] = df.groupby(by=["Country/Region","Province/State","Date","Case_Type"], as_index=False) \
        .agg({"Cases": "sum", "Long": "first", "Lat": "first"})

## Calculating case changes

Next, we sort the data by primary keys and `Date` to ensure we can add a `Differences` column as a window function.

In [ ]:
sorted_data = list(map(lambda df: df.sort_values(by=key_columns + ["Date"], ascending=True), unpivoted_data))

As `Cases` are actual snapshots (running numbers), we define changes as the difference to the previous day's value. In other words, `Difference` equals today's `Cases` minus yesterday's `Cases` for each region/state and each case category.

In [ ]:
for df in sorted_data:
    df["Difference"] = df["Cases"] - df.groupby( key_columns )["Cases"].shift(1, fill_value = 0) 

concated = pd.concat(sorted_data)

## Calculating active cases

Acive cases are confirmed cases that are not deceased or registered as recovered. This is relevant as active cases determine the demands on the healthcare system. We calculate `Active` case types as:

```
Active = Confirmed - Deaths - Recovered
```

As a first step, we merge the different type of cases into a single row for each `Country/Province/Date` keys:

In [ ]:
confirmed = concated[concated["Case_Type"].eq("Confirmed")]
deaths = concated[concated["Case_Type"].eq("Deaths")]
recovered = concated[concated["Case_Type"].eq("Recovered")]

active = confirmed  \
        .merge(deaths, validate= "one_to_one", suffixes =["","_d"], on=["Country/Region","Province/State","Date"]) \
        .merge(recovered, validate= "one_to_one", suffixes =["","_r"], on= ["Country/Region","Province/State","Date"])

Then, we apply the calculations both for `Cases` and `Difference`:

In [ ]:
active["Case_Type"] = "Active"
active["Cases"] = active["Cases"] - active["Cases_r"] - active["Cases_d"]
active["Difference"] = active["Difference"] - active["Difference_r"] - active["Difference_d"]

Finally, we merge the `Active` segment with the original one. 

In [ ]:
data = pd.concat([concated,active], join="inner")

data["Case_Type"].unique()

## Adding timestamp

Before we save the file locally, we add the `Last_Update_Date` in `UTC` time zone.

In [ ]:
data["Last_Update_Date"] = datetime.utcnow()

## Output

Finally, we store the output in the `output` folder as `JHU_COVID-19.csv` as an unindexed CSV file.

In [ ]:
data.to_csv(output_folder + "JHU_COVID-19.csv", index=False)